# update Jan 16th, 2022:
* Switch from using test chromedriver to webscraping for speed

In [200]:
import os
import sys
input_dir = os.getcwd()
# output_file = os.path.join(input_dir+ '\chromedriver.exe')
# print( output_file)/

import re
from bs4 import BeautifulSoup
import requests

import pandas as pd

In [201]:
#1. Cis, 2. Midaz, 3. Norepinephrin, 4. Dexmedotomidine, 5. Fentanyl, 6. Propofol
drugs = ['cisatracurium', 'midazolam', 'NOREPINEPHRINE','Dexmedetomidine', 'Fentanyl', 'propofol']

In [202]:
#define search root and link roots
root_url = "https://dailymed.nlm.nih.gov/dailymed/search.cfm?labeltype=all&query={}&pagesize=500&page=1"
link_root = "https://dailymed.nlm.nih.gov"


In [203]:
drugs_dict = {}
for d in drugs:
    r = requests.get(root_url.format(d.lower())) #connect to URL
    soup = BeautifulSoup(r.content, "html.parser") #parse HTML to BeautifulSoup object
    drugslinks = soup.find_all("a", {"class": "drug-info-link"}) #find all links with class "drug-info-link" 
    links = [link_root + link.get("href") for link in drugslinks] #get the href attribute of each link
    drugs_dict[d.upper()] = links


In [204]:
drugs_dict.keys()

dict_keys(['CISATRACURIUM', 'MIDAZOLAM', 'NOREPINEPHRINE', 'DEXMEDETOMIDINE', 'FENTANYL', 'PROPOFOL'])

In [205]:
#Count up the number of pages for each drug
for k, v in drugs_dict.items():
    print(k, len(v), " pages found")

CISATRACURIUM 19  pages found
MIDAZOLAM 72  pages found
NOREPINEPHRINE 200  pages found
DEXMEDETOMIDINE 50  pages found
FENTANYL 71  pages found
PROPOFOL 30  pages found


In [121]:
# Test parsing of a single link to obtain inactive ingredients (excipients)
r_indi = requests.get(links[1])
soup_indi = BeautifulSoup(r_indi.content, "html.parser")
# x = soup_indi.find_all("table",{"class":"formTablePetite"}) # Can be improved upon
x = soup_indi.find(string="Inactive Ingredients").find_parent("table") #find the table with the string "Inactive Ingredients" as caption and its parent
inactives = x.find_all("td", {"class":"formItem"})
excipients_names = [inactives[i].text.strip() for i in range(0,len(inactives),2)]
excipients_conc = [inactives[i].string.replace("\xa0","") for i in range(1,len(inactives),2)]
excipient_dict = dict(zip(excipients_names, excipients_conc))



## Get excipients Info

In [206]:
# look at individual links:
def get_inactives(urls_list):
    empty_list = []

    for l in urls_list:
        r_indi = requests.get(l)
        soup_indi = BeautifulSoup(r_indi.content, "html.parser")
        if soup_indi.find("li", {"class": "human"}) is not None: #check if human drugs
            try:
                x = soup_indi.find(string="Inactive Ingredients").find_parent("table") #find the table with the string "Inactive Ingredients" as caption and its parent
                
                inactives = x.find_all("td", {"class":"formItem"})
                excipients_names = [inactives[i].text.strip().upper() for i in range(0,len(inactives),2) if inactives[i].text.strip() != ""]
                excipients_conc = [inactives[i].string.replace("\xa0","") for i in range(1,len(inactives),2) if inactives[i].string is not None]
                excipient_dict = dict(zip(excipients_names, excipients_conc))

                empty_list.append(excipient_dict)
            except AttributeError:
                pass

        else:
            continue

    return empty_list



In [207]:
# Create a output file with the excipients for each drug
output_dict = {}
for k, v in drugs_dict.items():
    output_dict[k] = get_inactives(v)


In [208]:
# inspect result output:
for k, v in output_dict.items():
    print(k, len(v))

CISATRACURIUM 19
MIDAZOLAM 69
NOREPINEPHRINE 199
DEXMEDETOMIDINE 45
FENTANYL 67
PROPOFOL 28


In [209]:
with open("excipients_dict.txt", "w") as f:
    for k, v in output_dict.items():
        f.write(k + ": " + str(v) + "\n")

In [210]:
tes_list = []
for i in output_dict['PROPOFOL']:
    tes_list = list(set(tes_list + list(i.keys())))

In [211]:
# get unique excipients for each drug
drugs_dict_unique_excipients = {}

for k, v in output_dict.items():
    uni_ls = []
    for i in v:
        uni_ls = list(set(uni_ls + list(i.keys())))
    
    unii ={u.split(" (UNII:")[1].replace(")","").strip() : u.split(" (UNII:")[0] for u in uni_ls}

    drugs_dict_unique_excipients[k] = unii


In [212]:
# Write out look up table for excipients with its UNII
with open("excipients_lookups.txt", "w") as f:
    for k, v in drugs_dict_unique_excipients.items():
        f.write(k + ": " + str(v) + "\n")

In [213]:
# build summary table
drug_name = []
drug_excipients = []
drug_exp_conc = []
drug_configuration = []
for k, v in output_dict.items():
    config_idx = 0
    for d in v:
        drug_excipients = drug_excipients + list(d.keys())
        drug_exp_conc = drug_exp_conc + list(d.values())
        drug_name = drug_name + ([k]*len(list(d.keys())))
        drug_configuration = drug_configuration + ([config_idx]*len(list(d.keys())))
        config_idx += 1


# make data table from 4 lists
df = pd.DataFrame({'drug_name': drug_name, 'drug_excipients': drug_excipients, 'drug_exp_conc': drug_exp_conc, 'drug_configuration': drug_configuration})

# create column to store the UNII and get unique excipients
df['drug_excipients_unii'] = df['drug_excipients'].apply(lambda x: x.split(" (UNII:")[1].replace(")","").strip())
df['excipient_name_normalize'] = df.apply(lambda x: drugs_dict_unique_excipients[x['drug_name']][x['drug_excipients_unii']], axis=1)

In [214]:
#export pandas dataframe to csv
df.to_csv('excipients_table_17Jan22.csv', index=False)

In [215]:
df.head()

,drug_name,drug_excipients,drug_exp_conc,drug_configuration,drug_excipients_unii,excipient_name_normalize
0,CISATRACURIUM,BENZENESULFONIC ACID (UNII: 685928Z18A),,0,685928Z18A,BENZENESULFONIC ACID
1,CISATRACURIUM,WATER (UNII: 059QF0KO0R),,0,059QF0KO0R,WATER
2,CISATRACURIUM,BENZYL ALCOHOL (UNII: LKG8494WBH),,1,LKG8494WBH,BENZYL ALCOHOL
3,CISATRACURIUM,BENZENESULFONIC ACID (UNII: 685928Z18A),,1,685928Z18A,BENZENESULFONIC ACID
4,CISATRACURIUM,WATER (UNII: 059QF0KO0R),,1,059QF0KO0R,WATER


## Get Packaging Info

In [216]:
# look at individual links packaging:
def get_packaging(urls_list):
    ndcs = []
    pack_config = []
    date_released = []

    ingr_names = []
    basis_str = []
    strengths = []

    for l in urls_list:
        r_indi = requests.get(l)
        soup_indi = BeautifulSoup(r_indi.content, "html.parser")
        if soup_indi.find("li", {"class": "human"}) is not None: #check if human drugs
            try:
                x = soup_indi.find(string="Packaging").find_parent("table") #find the table with the string "Packaging" as caption and its parent
                
                inactives = x.find_all("td", {"class":"formItem"})

                ndcs.append([inactives[i].text.strip() for i in range(0,len(inactives),4) if inactives[i].text.strip() != ""])

                pack_config.append([inactives[i].string.replace("\xa0","") for i in range(1,len(inactives),4) if inactives[i].string is not None])

                date_released.append([inactives[i].string.replace("\xa0","") for i in range(2,len(inactives),4) if inactives[i].string is not None])

            except AttributeError:
                pass

            try:
                active_con = soup_indi.find(string="Active Ingredient/Active Moiety").find_parent("table") #find the table with the string "Active Ingredients" as caption and its parent

                actives = active_con.find_all("td", {"class":"formItem"})

                ingr_names.append([actives[i].text.strip() for i in range(0,len(actives),3) if actives[i].text.strip() != ""])

                basis_str.append([actives[i].string.replace("\xa0","") for i in range(1,len(actives),3) if actives[i].string is not None])

                strengths.append([actives[i].string.replace("\xa0","") for i in range(2,len(actives),3) if actives[i].string is not None])



            except AttributeError:
                pass

        else:
            continue

    return ndcs, pack_config, date_released, ingr_names, basis_str, strengths



In [217]:
#get all configurations for each drug
drug_name = []
drug_ndc = []
drug_pack_config = []
drug_date_released = []
drug_ingr_names = []
drug_basis_str = []
drug_strengths = []
for dr in drugs_dict:
    n, p, d, ing, b, s = get_packaging(drugs_dict[dr])
    drug_name = drug_name + [dr]*len(n)
    drug_ndc = drug_ndc + n
    drug_pack_config = drug_pack_config + p
    drug_date_released = drug_date_released + d
    drug_ingr_names = drug_ingr_names + ing
    drug_basis_str = drug_basis_str + b
    drug_strengths = drug_strengths + s



In [ ]:
# make data table from all lists
df_packing = pd.DataFrame({'drug_name': drug_name, 'drug_ndc': drug_ndc, 'drug_pack_config': drug_pack_config, 'drug_date_released': drug_date_released, 'drug_ingr_names': drug_ingr_names, 'drug_basis_str': drug_basis_str, 'drug_strengths': drug_strengths})

#export pandas dataframe to csv
df_packing.to_csv('drug_packaging_table_17Jan22.csv', index=False)

In [ ]:
# Write unique excipients to file:
with open("excipients_unique_17Jan22.txt", "w") as f:    
    for k, v in drugs_dict_unique_excipients.items():
        f.write(k + "~" + str(list(v.values())) + "\n")

# OLD VERSION USING WEBBROWSER BELOW

In [ ]:
search_url = "https://dailymed.nlm.nih.gov/dailymed/search.cfm?labeltype=all&query=CIPROFLOXACIN&pagesize=200&page=1"

In [ ]:
from selenium import webdriver
import pandas as pd
import time
import re

url = search_url
browser = webdriver.Chrome(executable_path=output_file)
browser.get(url)
browser.implicitly_wait(10)
articles = browser.find_elements_by_xpath("//div[@class='results']/article")

urls = []
for article in articles:
    url = article.find_element_by_class_name('drug-info-link').get_attribute('href')
    urls.append(url)

for url in urls:
    try:
        browser.get(url)
        browser.implicitly_wait(10)
        
        browser.find_element_by_id('anch_dj_109').click()
        time.sleep(1)
        
        li_tags = browser.find_elements_by_xpath("//div[@class='scrollingtable']/li")
        
        for li_tag in li_tags:
            txt = li_tag.text
            if 'INGREDIENTS AND APPEARANCE' in txt:
                tbs = li_tag.find_elements_by_class_name('internaltable')
                for tb in tbs:
                    product_type = ''
                    route_admin = ''
                    active_in = ''
                    strength = ''
                    inactives = ['', '', '', '', '', '', '', '', '', '']
                    inactive_cons = ['', '', '', '', '', '', '', '', '', '']
                    description = ''
                    tables = tb.find_elements_by_class_name('formTablePetite')
                    for table in tables:
                        trs = table.find_elements_by_tag_name('tr')
                        tbl_txt = table.text.lower()
                        if 'active ingredient/active moiety' in tbl_txt:
                            tr = trs[2]
                            tds = tr.find_elements_by_tag_name('td')
                            active_in = tds[0].text
                            strength = tds[2].text
                            continue
                        if 'package description' in tbl_txt:
                            cnt = -1
                            for tr in trs:
                                cnt += 1
                                if cnt < 2: continue
                                tds = tr.find_elements_by_tag_name('td')
                                description += tds[1].text + ';'
                                
                            continue
                        if 'inactive ingredients' in tbl_txt:
                            cnt = -1
                            for tr in trs:
                                cnt += 1
                                if cnt < 2: continue
                                tds = tr.find_elements_by_tag_name('td')
                                inactive = ''
                                con = ''
                                inactive = tds[0].text
                                con = tds[1].text
                                inactives[cnt - 2] = inactive
                                inactive_cons[cnt - 2] = con
                                
                            continue
                        if 'product information' in tbl_txt:
                            for tr in trs:
                                if 'Product Type' in tr.text:
                                    tds = tr.find_elements_by_tag_name('td')
                                    product_type = tds[1].text
                                elif 'Route of Administration' in tr.text:
                                    tds = tr.find_elements_by_tag_name('td')
                                    route_admin = tds[1].text
                            continue
                    try:
                        raw_data = {'product_type': [product_type],
                                    'route_admin': [route_admin],
                                    'active_in': [active_in],
                                    'strength': [strength],
                                    'inactive1': [inactives[0]],
                                    'inactive_con1': [inactive_cons[0]],
                                    'inactive2': [inactives[1]],
                                    'inactive_con2': [inactive_cons[1]],
                                    'inactive3': [inactives[2]],
                                    'inactive_con3': [inactive_cons[2]],
                                    'inactive4': [inactives[3]],
                                    'inactive_con4': [inactive_cons[3]],
                                    'inactive5': [inactives[4]],
                                    'inactive_con5': [inactive_cons[4]],
                                    'inactive6': [inactives[5]],
                                    'inactive_con6': [inactive_cons[5]],
                                    'inactive7': [inactives[6]],
                                    'inactive_con7': [inactive_cons[6]],
                                    'inactive8': [inactives[7]],
                                    'inactive_con8': [inactive_cons[7]],
                                    'inactive9': [inactives[8]],
                                    'inactive_con9': [inactive_cons[8]],
                                    'inactive10': [inactives[9]],
                                    'inactive_con10': [inactive_cons[9]],
                                    'description': [description]
                                    }
                        print('--', raw_data)
                        df = pd.DataFrame(raw_data,
                                          columns=['product_type', 'route_admin', 'active_in', 'strength',
                                                   'inactive1', 'inactive_con1', 'inactive2', 'inactive_con2',
                                                   'inactive3', 'inactive_con3', 'inactive4', 'inactive_con4', 'inactive5', 'inactive_con5', 'inactive6', 'inactive_con6', 'inactive7', 'inactive_con7', 'inactive8', 'inactive_con8', 'inactive9', 'inactive_con9', 'inactive10', 'inactive_con10', 'description'])
                        df.to_csv('Cipro_25May21.csv', mode='a', header=False, index=False)

                    except Exception as ex:
                        print(ex)
    except Exception as ex:
        print(ex)

browser.quit()

In [ ]:
getdata(testtxt, "sodium chloride")